# Databases assignment 12

In [150]:
%%bash
docker run \
    -d --name neo4j \
    --rm \
    --publish=7474:7474 \
    --publish=7687:7687 \
    --env NEO4J_AUTH=neo4j/fancy!99Doorknob \
    --cpus=2 \
    --memory=8192m \
    -v "$(pwd):/var/lib/neo4j/import" \
    neo4j

4849a02d17c29a9cb42364ed1b1456f4144504b2dbf36bbc69a9c9cad609037d


In [162]:
import sys
from neo4j import GraphDatabase


uri = "bolt://localhost:7687"
auth=("neo4j", "fancy!99Doorknob")
driver = GraphDatabase.driver(uri, auth=auth)


def neo(command):
    try:
        with driver.session() as session:
            result = session.run(command)
        return result # result is a resultset/cursor for neo4j
    except Exception as ex:
        print(str(ex), file=sys.stderr)
        
        
def neov(command):
    try:
        return neo(command).values()
    except Exception as ex:
        return 'Shit happened'
"done"

'done'

## Exercise 1

In [175]:
neov("""
LOAD CSV WITH HEADERS FROM "file:///some2016UKgeotweets.csv" AS row FIELDTERMINATOR ";"
CREATE (t:Tweet { username: row.`User Name`, 
                 nickname: row.Nickname, 
                 tweetContent: row.`Tweet Content`, 
                 latitude: toFloat(row.Latitude), 
                 longitude: toFloat(row.Longitude), 
                 place: row.`Place (as appears on Bio)`})
""")

[]

In [177]:
neov("""
MATCH (a:Tweet)
WITH EXTRACT(m IN FILTER(m IN SPLIT(a.tweetContent," ") 
                  WHERE m STARTS WITH "@" AND SIZE(m) > 1) | RIGHT(m, size(m) - 1))
                  AS mentions, a
SET a.mentions = mentions
""")

[]

## Exercise 2

In [178]:
neov("""
MATCH (tweet:Tweet) 
WITH DISTINCT(tweet.username) AS username
CREATE (tweeter:Tweeter { username: username })
""")

[]

In [179]:
neov("""
MATCH (tweet:Tweet)
UNWIND tweet.mentions AS mentions
MATCH (tweeter:Tweeter { username: mentions })
CREATE (tweet)-[:MENTIONS]->(tweeter)
""")

[]

In [180]:
neov("""
MATCH (tweet:Tweet), (tweeter:Tweeter)
WHERE tweet.username = tweeter.username
CREATE (tweeter)-[:TWEETED]->(tweet)
""")

[]

## Exercise 3

Couldn't get anything to work here 😵